## Step 1: Data Preparation

In [1]:
!pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/c6/51/f140402202af6ce1bf747243f66415c5eb2f43ba2e2ac419a7e855d20673/streamlit-1.36.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/bb/2a/10164ed1f31196a2f7f3799368a821765c62851ead0e630ab52b8e14b4d0/blinker-1.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/04/e6/a1551acbaa06f3e48b311329828a34bc9c51a8cfaecdeb4d03c329a1ef85/cachetools-5.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<6,>=3.20 from https://files.pythonhosted.org/packages/b1/04/73b8fd7f34f3a2b2b64aa31a173b8aebbdb0c55523df4c027846bb44bc1e/protobuf-5.27.2-cp310-abi3-win_amd64.whl.metadata
  Obtaining dependency information for gitpython!=3.1.19,<4,>=3.0.7 from https://files.pythonhosted.org/packages/e9/bd/cc3a402a6439c15c3d4294333e13042b

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset from CSV file
df = pd.read_csv('tic-tac-toe.csv')

# Encode the features
le = LabelEncoder()
for col in df.columns:
    df[col] = le.fit_transform(df[col])

# Split the dataset
X = df.drop('class', axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Step 2: Model Training

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 87.50%


## Step 3: Creating the Streamlit App

In [4]:
# Save the trained model
import joblib
joblib.dump(model, 'tic_tac_toe_model.pkl')

['tic_tac_toe_model.pkl']

## Step 4: Creating the Streamlit App

In [5]:
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model = joblib.load('tic_tac_toe_model.pkl')

st.title('Tic-Tac-Toe with AI')
st.write('Play against the AI!')

# Initialize board state
if 'board' not in st.session_state:
    st.session_state.board = [''] * 9
    st.session_state.current_player = 'X'

# Reset game function
def reset_game():
    st.session_state.board = [''] * 9
    st.session_state.current_player = 'X'

# Handle cell click
def handle_click(index):
    if st.session_state.board[index] == '':
        st.session_state.board[index] = st.session_state.current_player
        if check_winner(st.session_state.board, st.session_state.current_player):
            st.success(f'Player {st.session_state.current_player} wins!')
            st.session_state.current_player = None
        elif '' not in st.session_state.board:
            st.info('It\'s a draw!')
            st.session_state.current_player = None
        else:
            st.session_state.current_player = 'O' if st.session_state.current_player == 'X' else 'X'
            if st.session_state.current_player == 'O':
                ai_move()

# AI Move function
def ai_move():
    board_state = np.array(st.session_state.board)
    board_state[board_state == ''] = 'b'
    board_state[board_state == 'X'] = 'x'
    board_state[board_state == 'O'] = 'o'
    board_encoded = le.transform(board_state)
    pred = model.predict([board_encoded])
    for i in range(9):
        if st.session_state.board[i] == '' and pred[0] == True:
            st.session_state.board[i] = 'O'
            break
    if check_winner(st.session_state.board, 'O'):
        st.success('AI wins!')
        st.session_state.current_player = None
    elif '' not in st.session_state.board:
        st.info('It\'s a draw!')
        st.session_state.current_player = None

# Check winner function
def check_winner(board, player):
    winning_conditions = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],
        [0, 3, 6], [1, 4, 7], [2, 5, 8],
        [0, 4, 8], [2, 4, 6]
    ]
    for condition in winning_conditions:
        if all(board[i] == player for i in condition):
            return True
    return False

# Display the board
st.write('Current Player: ', st.session_state.current_player)
for i in range(3):
    cols = st.columns(3)
    for j in range(3):
        index = i * 3 + j
        cols[j].button(st.session_state.board[index], key=index, on_click=handle_click, args=(index,))

# Reset game button
if st.button('Reset Game'):
    reset_game()


2024-07-17 19:16:16.310 
  command:

    streamlit run C:\Users\aksha\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-07-17 19:16:16.311 Session state does not function when running a script without `streamlit run`
